In [1]:
from pyhdf.HDF import HDF
from pyhdf.VS import VS
from pyhdf.SD import SD, SDC
import numpy as np
from datetime import datetime, timezone, timedelta
import timeit
import os

os.environ["TZ"] = "UTC"

iso8601_string = "seconds since 1970-01-01T00:00:00Z"
epoch = datetime.fromisoformat(iso8601_string[14:-1])
epoch = epoch.replace(tzinfo=timezone.utc)

niters = 1000000

In [13]:
def get_normalized_bit(value, bit_index):
    return (value >> bit_index) & 1

In [3]:
wbeg = np.datetime64(datetime.strptime("200101010000", "%Y%m%d%H%M"))
wend = np.datetime64(datetime.strptime("210001010000", "%Y%m%d%H%M"))

In [2]:
# infile = '/glade/work/swei/data/RawOBS/CALIPSO/test_subset.hdf'
# infile = '/glade/work/swei/skylab/devtmp/iodaconv/test/testinput/caliop_l2_APro.hdf'
infile = '/glade/work/swei/data/RawOBS/CALIPSO_v451/CAL_LID_L2_05kmAPro-Standard-V4-51.2019-07-22T03-45-56ZN_Subset.hdf'
# infile = '/glade/work/swei/data/RawOBS/CALIPSO/CAL_LID_L1-Standard-V4-10.2019-07-22T03-45-56ZN.hdf'
hdf = HDF(infile)
vs = hdf.vstart()
metaid = vs.find('metadata')
vd = vs.attach(metaid)
vd.setfields('Lidar_Data_Altitudes')
alt = np.array(vd.read()[0][0]) * 1000
nlev = alt.size
vd.detach()
vs.end()

In [15]:
sd = SD(infile, SDC.READ)
caliop_ref_time = datetime(1993, 1, 1, 0, 0, 0)

In [5]:
thickness = np.empty_like(alt)
thickness[1:-1] = 0.5 * (alt[:-2] - alt[2:])
thickness[0] = alt[0] - alt[1]
thickness[-1] = alt[-2] - alt[-1]

In [10]:
def calipso_time2dt(time):
    dtarr = [datetime.strptime(str(t)[:6], '%y%m%d') for t in time]
    delta = [timedelta(frac) for frac in np.mod(time, 1)]
    outarr = [(dt + dl).replace(tzinfo=timezone.utc) for dt, dl in zip(dtarr, delta)]
    return outarr

In [78]:
utcproftime = sd.select('Profile_UTC_Time').get()[:, 1]
utcobs_time = np.array([(pt - epoch).total_seconds() for pt in calipso_time2dt(utcproftime)], dtype=np.int64)

In [28]:
proftime = sd.select('Profile_Time').get()[:, 1]
obs_time = np.array([(proftime + caliop_ref_time.timestamp()).total_seconds()])

AttributeError: 'numpy.ndarray' object has no attribute 'total_seconds'

In [25]:
pres = sd.select('Pressure').get().ravel() * 1e2
cad1 = np.zeros_like(pres)
cad2 = np.zeros_like(pres)
tmpcad = sd.select("CAD_Score").get()
tmpcad = tmpcad.reshape(-1, tmpcad.shape[2])
cad1 = tmpcad[:, 0]
cad2 = tmpcad[:, 1]

In [31]:
cad1[9975:9978]

array([-127, -127, -127], dtype=int8)

In [32]:
oricad = sd.select("CAD_Score").get()

In [33]:
oricad.shape

(50, 399, 2)

In [64]:
pres = sd.select('Pressure').get().ravel() * 1e2  # hPa to Pa
lats = sd.select('Latitude').get()[:, 1]
nloc = lats.size
lats = np.repeat(lats, nlev)
lons = sd.select('Longitude').get()[:, 1]
lons = np.repeat(lons, nlev)
profidx = np.arange(lats.size)
proftime = sd.select('Profile_Time').get()[:, 1]
obs_time = (proftime + caliop_ref_time.timestamp()).astype('datetime64[s]')
obs_time = np.tile(obs_time, nlev)
winmsk = ((obs_time >= wbeg) & (obs_time <= wend))

In [65]:
lats

array([58.274254, 58.274254, 58.274254, ..., 56.133595, 56.133595,
       56.133595], dtype=float32)

In [52]:
cad = sd.select("CAD_Score").get()

In [54]:
cad.shape

(50, 399, 2)

In [36]:
caliop_ref_time = datetime(1993, 1, 1, 0, 0, 0)
proftime = sd.select('Profile_Time').get()[:, 1]
obs_time = (proftime + caliop_ref_time.timestamp()).astype('datetime64[s]')
obs_time = np.tile(obs_time[:, np.newaxis], (1, 399))

In [38]:
obs_time.shape

(50, 399)

In [21]:
qc1 = sd.select('QC_Flag').get()
qc2 = sd.select('QC_Flag_2').get()

In [24]:
a = sd.select('Total_Attenuated_Backscatter_532').get()

In [15]:
bit_list1_532 = [[1, 2], 4, 5, 7, [8, 9], [11, 12], 13, 15, 17, 19, 20, 22, 24]
bit_list2_532 = [[2, 3], [5, 6], [8, 9], [11, 12], [14, 15], [17, 18], [20, 21], [23, 24], 26]
bit_list_1_1064 = [3, 4, 6, 10, 14, 16, 18, 21, 23]
bit_list_2_1064 = [4, 7, 10, 13, 16, 19, 22, 25, 26, 27]

In [26]:
qc_bit_dict = {
    '532': {
        'region1': [1, 2]
        'region2': 
    }
    '1064':
}

SyntaxError: '{' was never closed (1848822094.py, line 1)

In [3]:
# Level 2 extinction
sd = SD(infile, SDC.READ)
data = sd.select('Extinction_Coefficient_532').get()
data2 = sd.select('Extinction_Coefficient_1064').get()
err = sd.select('Extinction_Coefficient_Uncertainty_532').get()
cad = sd.select('CAD_Score').get()

In [8]:
data[data != -9999.0].max()

18.173855

In [16]:
sd.select('CAD_Score').dimensions()

{'fakeDim75': 4192, 'fakeDim76': 399, 'fakeDim77': 2}

In [41]:
mask = data=-9999.
mask2 = (data2<0.)&(data2!=-9999.)

In [25]:
c = np.stack((data, data2), axis=1)

In [26]:
c.shape

(4192, 2, 399)

In [31]:
lats=hdf.select('Latitude').get()[:,1]
lats.shape

(4192,)

In [79]:
qcf = sd.select('Extinction_QC_Flag_1064').get()
tmpqcf = np.where(qcf[:, :, 0]==qcf[:, :, 1], qcf[:, :, 0], qcf.sum(axis=2))
tmpqcf.shape

(4192, 399)

In [80]:
qcf.shape

(4192, 399, 2)